# 2022年5月16日 Julia 学习笔记
## 设计抽象类型和具体类型
### 设计抽象类型
父类型-子类型，术语：超类型-子类型
抽象类型仅用于为一组对象的行为建模，而不用与指定数据的存储方式。
创建抽象类型层次结构
- 资产Asset
  - 财产Property
    - 房屋House
    - 公寓Apartment
  - 投资Investment
      - 固定收益FixedIncome
      - 股权Equity
  - 现金Cash
上述结构，使用以下代码创建：

In [39]:
abstract type Asset end

abstract type Property <: Asset end
abstract type Investment <: Asset end
abstract type Cash <: Asset end

abstract type House <: Property end
abstract type Apartment <: Property end

abstract type FixedIncome <: Investment end
abstract type Equity <: Investment end

<:符号表示超类型-子类关系，因此Property是Asset的子类型，Equity是Investment的子类。事实上，Asset有一个称为Any的超类型，Any是Julia中的顶级超类型。Julia提供了一些方便的函数来导航类型层次结构，比如subtypes函数。

In [40]:
subtypes(Asset)

3-element Vector{Any}:
 Cash
 Investment
 Property

In [41]:
supertype(Equity)

Investment

以树形格式查看完整层次结构，没有提供现成的标准函数，可以自己用递归创建一个，如下所示

In [42]:
function subtypetree(roottype,level = 1, indent = 4)
    level == 1 && println(roottype)
    for s in subtypes(roottype)
        println(join(fill(" ", level * indent)) * string(s))
        subtypetree(s, level + 1, indent)
    end
end

subtypetree (generic function with 3 methods)

In [43]:
subtypetree(Asset)

Asset
    Cash
    Investment
        Equity
            Stock
            mStock
        FixedIncome
    Property
        Apartment
        House


#### 定义抽象类型的函数
##### 描述函数

In [44]:
# simple functions on abstract types
describe(a::Asset) = "Something valuable"
describe(e::Investment) = "Finacial investment"
describe(e::Property) = "Physical property"

describe (generic function with 4 methods)

由于尚未定义任何具体类型，因此无法证明Cahs对象的describe函数将使用describe(a::Asset)方法。

##### 函数行为

具有层次结构的原因是为了创建有关类型的常见行为的抽象。例如：Apartment和House类型具有相同的超类型Property。有意这么继承是因为他们都代表特定位置的某种物理住所。因此可以定义函数如下：

In [45]:
"""
location(p::Property)
Returns the location of the property as a tuple of (latitude, longitude).
"""
location(p::Property) = error("Location is not defined in the concrete type")

location

上述函数只实现了返回错误一个功能，定义此函数的实际目的有以下：
- Property的任何具体子类型都必须实现location函数
- 如果没有为相应的具体类型定义location函数，则运行时将调用此特定函数并抛出合理错误，一边后续修改bug
- 函数定义上方的文档字符串包含有用的描述，即Property的具体子类型实现。
另外，还可以定义一个空函数：

In [46]:
"""
location(p::Property)
Returns the location of the property as a tuple of (latitude, longitude).
"""
function location(p::Property) end

location

空函数，如果具体类型不实现该函数，则不会出现运行时错误。
##### 对象之间的相互作用

知道了每个Property都有一个位置，可以定义一个函数来计算两个财产之间的步行距离：

In [47]:
function walking_distance(p1::Property, p2::Property)
    loc1 = location(p1)
    loc2 = location(p2)
    return abs(loc1.x - loc2.x) + abs(loc1.y - loc2.y)
end

walking_distance (generic function with 1 method)

可以看到，逻辑完全存在于抽象类型中，我们甚至都没有定义任何具体类型，但是能够开发通用代码，适用于以后的Property任何具体子类型。

### 设计具体类型
具体类型用于定义数据的组织方式，有以下两种：
- 原始类型
- 复合类型

复合类型由一组命名字段定义。将字段分组为单一类型可简化推理、共享和操作。

**复合类型**可以指定为特定的超类型，也可以默认为Any。

若需要，还可以用字段自己的类型来注释字段，并且类型可以是抽象的也可以是具体的。如果缺少字段类型信息，则默认为Any，表示该字段可以容纳任何类型的对象。

#### 设计复合类型
复合类型使用struct关键字定义。
比如，创建以下类型Stock来表示股票的交易代码(symbol)和名称(name):

In [48]:
struct Stock <: Equity 
    symbol::String
    name::String
end

可以使用标准构造函数实例化复合类型，该构造函数将所有字段用作参数：

In [49]:
stock = Stock("APPL", "Apple, Inc.")

Stock("APPL", "Apple, Inc.")

In [50]:
describe(stock)

"APPL(Apple, Inc.)"

可以看到，缺省情况下，Stock由超类型Equity的超类型Investment来调用。可以通过定义describe函数来实现具体描述：

In [51]:
function describe(s::Stock)
    return s.symbol * "(" * s.name * ")"
end

describe (generic function with 4 methods)

In [52]:
describe(stock)

"APPL(Apple, Inc.)"

再次运行describe可以发现函数返回结果已经改变。

##### 不可变性

不可变性消除了由于数据修改而导致系统行为意外更改时的意外情况。比如下面这样的程序，会报错。

In [53]:
stock.name = "Apple LLC"

LoadError: setfield!: immutable struct of type Stock cannot be changed

不可变性实际上最多保证到字段级别，如果某个类型包含一个字段，并且该字段自己的类型是可变的，则允许更改基础数据。比如，创建一个成为BasketOfStocks的新复合类型，用于保存股票数组及持有股票的原因：

In [ ]:
struct BasketOfStocks
    stocks::Vector{Stock}
    reason::String
end

In [ ]:
many_stocks = [
    Stock("APPL", "Apple, Inc."),
    Stock("IBM", "IBM")
]
basket = BasketOfStocks(many_stocks, "Anniversary gift for my wife")

BasketOfStocks(Stock[Stock("APPL", "Apple, Inc."), Stock("IBM", "IBM")], "Anniversary gift for my wife")

In [ ]:
pop!(basket.stocks)

Stock("IBM", "IBM")

In [ ]:
basket

BasketOfStocks(Stock[Stock("APPL", "Apple, Inc.")], "Anniversary gift for my wife")

可以发现，函数可以直接作用与stock对象，因此**不可变性对基础字段是没有任何影响的**。
##### 可变性
在某些情况下，我们实际上可能希望对象是可变的。只需在类型定义前面添加mutable关键字，即可轻松取消的不可变性的约束。比如，为了使Stock类型可变，可以执行以下操作：

In [ ]:
mutable struct mStock <: Equity 
    symbol::String
    name::String
end

In [ ]:
mstock = mStock("APPL", "Apple, Inc.")

mStock("APPL", "Apple, Inc.")

In [ ]:
mstock.name = "Apple LLC"

"Apple LLC"

In [ ]:
mstock

mStock("APPL", "Apple LLC")

##### 可变还是不可变

通常情况下不希望默认可变，主要原因：
1. 不可变的对象更易于处理。
2. 可变对象在多线程应用程序中更难以使用。

##### 使用Union类型支持多种类型
当需要合并来自不同数据类型层次结构的数据类型时，Union类型非常有用。

In [ ]:
abstract type Art end

struct Painting <: Art
    artist::String
    title::String
end

可以将Stock和Painting两种类型进行组合，如下所示：

In [ ]:
struct BasketOfThings
    things::Vector{Union{Painting, Stock}}
    reason::String
end

In [ ]:
stock = Stock("APPL", "Apple, Inc.")

Stock("APPL", "Apple, Inc.")

In [ ]:
monalisa = Painting("Leonardo da Vinci", "Monalisa")

Painting("Leonardo da Vinci", "Monalisa")

In [ ]:
things = Union{Painting, Stock}[stock, monalisa]

2-element Vector{Union{Painting, Stock}}:
 Stock("APPL", "Apple, Inc.")
 Painting("Leonardo da Vinci", "Monalisa")

In [ ]:
present = BasketOfThings(things, "Anniversary gift for my wife")

BasketOfThings(Union{Painting, Stock}[Stock("APPL", "Apple, Inc."), Painting("Leonardo da Vinci", "Monalisa")], "Anniversary gift for my wife")

Union类型的语法可能非常冗长，尤其是存在两种以上类型时，因此使用定义了代表Union类型的有意义名称的常量是很常见的：

In [ ]:
const Thing = Union{Painting, Stock}

struct tBasketOfThings
    thing::Vector{Thing}
    reason::String
end

Thing比Union{Painting, Stock}更容易阅读，并且可以在很多地方引用，当后续需要添加更多类型时，只需要在定义处更改一次，因此维护代码可以更加轻松。

### 使用类型运算符
#### isa运算符
isa运算符可以用于确定某个值是否是类型的子类型，比如：

In [ ]:
1 isa Int

true

In [ ]:
1 isa Real

true

In [ ]:
1 isa Float64

false

isa运算符对于检查接受泛型类型参数的函数中的类型很有用。
#### <:运算符
用于确定某个类型是否为另一种类型的子类型。

In [ ]:
Int <: Real

true

isa根据类型检查一个变量是否是该类型，检查的对象是变量。
<:检查的对象仍然是类型。

1. 可以用于检验传递参数有无正确类型。
2. 根据传递给函数的参数类型动态地执行不同逻辑。

### 抽象类型和具体类型的差异

| 问题 | 抽象类型 | 具体类型 |
|------|---------|---------|
| 有子类型么？| 有 | 有 |
|允许子类型么？| 允许 | 不允许 |
|包含数据字段么？| 不包含 | 包含 |
| 第一类实体？ | 是 | 是 |
|可以作为Union类型么？| 可以 | 可以 |


抽象类型表示概念，而不是数据存储。是第一类实体的，可以存储和传递，并且可以使用它们的函数。

具体类型与抽象类型作为超类型相关联。如果未指定超类型，则假定为Any。具体类型不允许子类型，因此每个具体类型必须是最终类型，并且将是类型层次结构中的叶节点。

Union类型可以引用抽象类型和具体类型。